In [1]:
# 75.06 - 1C202 - TP2 
# Competencia de Machine learning
#
# https://www.kaggle.com/c/nlp-getting-started/overview
#
# Fuentes:
# https://realpython.com/python-keras-text-classification/
# https://towardsdatascience.com/data-augmentation-in-nlp-2801a34dfc28

# Importacion de librerias necesarias
import re, string, random
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# NLTK
# https://www.nltk.org
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import TweetTokenizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer 

# NLPAUG
# https://github.com/makcedward/nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

# Split y K-Fold 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Importacion de Keras
# https://keras.io
from keras.models import Sequential, save_model, load_model
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Configuracion general
plt.style.use('ggplot')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pato\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Pato\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pato\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pato\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Seccion de CONFIGURACION

# Path de entrada
train_path = 'data/train.csv'
test_path = 'data/test.csv'

# Configuracion del modelo
maxlen = 100
num_filters = 32
kernel_size = 8
embedding_dim = 200
embeddings_path = 'embeddings/glove.twitter.27B.200d.txt'
# Embeddings disponibles en:
# https://worksheets.codalab.org/bundles/0x4090ba96b8a444c2a44b2c47884c25f2

# Configuracion de entrenamiento
num_folds = 3
epochs = 6
verbose = False
batch_size = 10

# Path de para guardar los modelos
model_path = 'models/TP2.Keras.Conv1D.Iter.'

# Path de salida para el submission
submission_path = 'data/submits/submission_Deep_Conv1D_nltk_10.csv'

In [3]:
# Funciones AUXILIARES

# Pasa un texto a lowercase
def lowercase_text(text):
    return text.lower()

# Limpia los datos, mediante el uso de expresiones regulares
def remove_noise(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Grafica para un modelo los resultados del entrenamiento
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Entrenamiento')
    plt.plot(x, val_acc, 'r', label='Validación')
    plt.title('Accuracy: validación y entrenamiento')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Entrenamiento')
    plt.plot(x, val_loss, 'r', label='Validación')
    plt.title('Loss: validación y entrenamiento')
    plt.legend()
    plt.show()

# Genera la matriz de embeddings
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # 0 es un indice reservado, sumamos 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding='utf-8') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

# Define el modelo a utilizar, con Conv1D
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [4]:
# Carga de los archivos del set de datos

# Set de train: carga, descarte de columnas no utilizadas y limpieza de los textos
df_twitter_train = pd.read_csv(train_path, sep=',')
#Sumo los keywords al text
df_twitter_train['keyword'].fillna(' ', inplace=True)
df_twitter_train['text'] = (df_twitter_train['keyword'] + ' ' + df_twitter_train['text'])
df_twitter_train.drop('location', axis=1, inplace=True)
df_twitter_train['text'] = df_twitter_train['text'].apply(lambda x: lowercase_text(x))
df_twitter_train['keyword'] = df_twitter_train['keyword'].apply(lambda x: lowercase_text(x))


# Set de test: carga, descarte de columnas no utilizadas y limpieza de los textos
df_twitter_test = pd.read_csv(test_path, sep=',')
#Sumo los keywords al text
df_twitter_test['keyword'].fillna(' ', inplace=True)
df_twitter_test['text'] = (df_twitter_test['keyword'] + ' ' + df_twitter_test['text'])
df_twitter_test.drop('location', axis=1, inplace=True)
df_twitter_test['text'] = df_twitter_test['text'].apply(lambda x: lowercase_text(x))
df_twitter_test['text'] = df_twitter_test['text'].apply(lambda x: remove_noise(x))
df_twitter_test['keyword'] = df_twitter_test['keyword'].apply(lambda x: lowercase_text(x))


# Vemos el tamaño de ambos sets
print(df_twitter_train.shape)
print(df_twitter_test.shape)

(7613, 4)
(3263, 3)


In [5]:
df_twitter_train.head()

,id,keyword,text,target
0,1,,our deeds are the reason of this #earthquake...,1
1,4,,forest fire near la ronge sask. canada,1
2,5,,all residents asked to 'shelter in place' ar...,1
3,6,,"13,000 people receive #wildfires evacuation ...",1
4,7,,just got sent this photo from ruby #alaska a...,1


In [6]:
# Tratamiento del set con nltk y nlpaug
# NLTK
# Sabiendo que todos los tweets son en idioma ingles, quitamos las stopwords
stop_words = set(stopwords.words('english'))
df_twitter_train['text'] =  df_twitter_train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_twitter_test['text'] =  df_twitter_test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_twitter_train['keyword'] =  df_twitter_train['keyword'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_twitter_test['keyword'] =  df_twitter_test['keyword'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))



# NLPAUG
# Tratamiento del set de train con nlpaug
# Aumentamos el tamaño de nuestro set para mejorar los resultados de la prediccion

# Creamos un nuevo dataframe para trabajar
df_twitter_train_aug = pd.DataFrame(df_twitter_train)

# Reemplazamos una palabra del tweet con un sinonimo tomado de WordNet
aug_syn = naw.SynonymAug(aug_src='wordnet')
# Intercambiamos una palabra random por otra en el tweet
aug_swp = naw.RandomWordAug(action="swap")

# Usamos las 2 opciones
#df_twitter_train_aug['text'] = df_twitter_train_aug['text'].apply(lambda x: aug_syn.augment(x))
#df_twitter_train_aug['text'] = df_twitter_train_aug['text'].apply(lambda x: aug_swp.augment(x))

# Unimos los 2 dataframes
#df_twitter_train_aug = pd.concat([df_twitter_train, df_twitter_train_aug])

# Stemmizamos los textos de los tweets
#ps = PorterStemmer()
#df_twitter_train_aug['text'] =  df_twitter_train_aug['text'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))
#df_twitter_test['text'] =  df_twitter_test['text'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))

# Lematizamos los textos de los tweets
lem = WordNetLemmatizer()
df_twitter_train['text'] =  df_twitter_train['text'].apply(lambda x: ' '.join([lem.lemmatize(word,"v") for word in x.split()]))
df_twitter_test['text'] =  df_twitter_test['text'].apply(lambda x: ' '.join([lem.lemmatize(word,"v") for word in x.split()]))
df_twitter_test['keyword'] =  df_twitter_test['keyword'].apply(lambda x: ' '.join([lem.lemmatize(word,"v") for word in x.split()]))
df_twitter_train['keyword'] =  df_twitter_train['keyword'].apply(lambda x: ' '.join([lem.lemmatize(word,"v") for word in x.split()]))



# Vemos el tamaño final del set de train
print(df_twitter_train_aug.shape)

(7613, 4)


In [7]:
# Vemos como quedo el set de train
df_twitter_train_aug.head()

,id,keyword,text,target
0,1,,deeds reason #earthquake may allah forgive us,1
1,4,,forest fire near la ronge sask. canada,1
2,5,,residents ask 'shelter place' notify officers....,1
3,6,,"13,000 people receive #wildfires evacuation or...",1
4,7,,get send photo ruby #alaska smoke #wildfires p...,1


In [8]:
# Vemos como quedo el set de test
df_twitter_test.head()

,id,keyword,text
0,0,,happen terrible car crash
1,2,,hear earthquake different cities stay safe eve...
2,3,,forest fire spot pond geese flee across street...
3,9,,apocalypse light spokane wildfires
4,11,,typhoon soudelor kill china taiwan


In [9]:
# Nos quedamos con X e y para entrenar
tweets = df_twitter_train_aug['text'].values
y = df_twitter_train_aug['target'].values

# Separamos X para el set de test
tweets_predict = df_twitter_test['text'].values
keyword_predict = df_twitter_test['keyword'].values

In [10]:
# Tokenizamos los textos
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweets)

# Convertimos a secuencias
X = tokenizer.texts_to_sequences(tweets)
X_predict = tokenizer.texts_to_sequences(tweets_predict)

# Calculamos el tamaño
vocab_size = len(tokenizer.word_index) + 1  # 0 es un indice reservado, sumamos 1

#PROBAR CON EL TOKENIZADOR DE NTK DE TWITTER

In [11]:
# Paddeamos a maxlen
X = pad_sequences(X, padding='post', maxlen=maxlen)
X_predict = pad_sequences(X_predict, padding='post', maxlen=maxlen)

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

In [40]:
# Generacion del dataframe que generara el submit para la competencia
kaggle_submission = pd.DataFrame(df_twitter_test,columns = ['id'])
kaggle_submission['target'] = y_predict
kaggle_submission.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [41]:
# Escritura del submit en formato CSV
kaggle_submission.to_csv(submission_path, index=False)

In [33]:
# ToDo:
# Grid search, busqueda de hiper-parametros ideales